In [ ]:
%matplotlib notebook
import mahotas as mh
import matplotlib.pyplot as plt
import numpy as np
from os.path import join
import ipywidgets as widgets
import cv2
from czifile import CziFile

path = "/home/phuijse/Work/Data/Araya/"

with CziFile(join(path, "test2.czi")) as czi:
    image_arrays = czi.asarray()
    meta = czi.metadata(raw=False)
    
#display(meta)

OpenCV and Mahotas

- http://www.cmm.mines-paristech.fr/~beucher/wtshed.html
- https://docs.opencv.org/master/d3/db4/tutorial_py_watershed.html
- https://github.com/luispedro/python-image-tutorial/blob/master/Segmenting%20cell%20images%20(fluorescent%20microscopy).ipynb
- https://docs.opencv.org/3.4/d1/d79/group__photo__denoise.html#ga21abc1c8b0e15f78cd3eff672cb6c476


TODO
- Calibrate h, templateWidowSize and serachWindowSize  (Denoising)
- Calibrate threshold (Thresholding)
- Understand how distances are calculated and what is the watershed marking

In [ ]:
fig, ax = plt.subplots(2, 4, figsize=(9, 5), tight_layout=True, sharex=True, sharey=True)

def update(k, z, h):
    img = image_arrays[0, 0, 0, k, z, :, :, 0]
    img = ((img/img.max())*255).astype('uint8')
    ax[0, 0].imshow(img, cmap=plt.cm.Greys_r)
    # Denoising
    img = cv2.medianBlur(img, 5)
    ax[0, 1].imshow(img, cmap=plt.cm.Greys_r)
    dst = cv2.fastNlMeansDenoising(img, None, h=h, templateWindowSize=7, searchWindowSize=21)
    ax[0, 2].imshow(dst, cmap=plt.cm.Greys_r)
    # Adaptive thresholding
    binary_mask = cv2.adaptiveThreshold(dst, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 21, 0)
    ax[0, 3].imshow(binary_mask, cmap=plt.cm.Greys_r)
    # Morphological operations on mask
    binary_mask = cv2.morphologyEx(binary_mask, cv2.MORPH_CLOSE, 
                                   kernel=np.ones((3,3),np.uint8), iterations = 4)
    ax[1, 0].imshow(binary_mask, cmap=plt.cm.Greys_r)
    # Local minima
    #ret, sure_fg = cv2.threshold(dist, 1, 255,0)
    #ret, markers = cv2.connectedComponents(sure_fg)
    minima = mh.regmin(binary_mask)
    markers, nr_markers = mh.label(minima)
    dst2 = dst.copy()
    dst2[minima] = 255
    ax[1, 1].imshow(dst2, cmap=plt.cm.Greys_r)
    # watershed
    watershed = cv2.watershed(cv2.cvtColor(dst, cv2.COLOR_GRAY2BGR), markers)
    ax[1, 2].imshow(watershed, cmap=plt.cm.tab20)
    # Lines from watershed
    dst3 = dst.copy()
    dst3[watershed==-1] = 255
    ax[1, 3].imshow(dst3, cmap=plt.cm.Greys_r)

widgets.interact(update, 
                 k=widgets.IntSlider(min=0, max=59),
                 z=widgets.IntSlider(min=0, max=16),
                 h=widgets.FloatSlider(min=0., max=100., value=30, step=0.01)
                 );